In [1]:
import numpy as np
import pandas as pd 
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from imutils import paths
import os

In [2]:
path = "D:/FlipKart_GRID/archive/data/New folder"
LABELS = list(['b1', 'm1'])
paths_ = []
for i in LABELS:
  paths_.append(path+'/'+i)
paths_

['D:/FlipKart_GRID/archive/data/New folder/b1',
 'D:/FlipKart_GRID/archive/data/New folder/m1']

In [3]:
imagepaths = []
data = []
labels = []
for i in range(len(paths_)):
  imagepaths = list(paths.list_images(paths_[i]))
  for imagepath in imagepaths:
    #print(imagepath)
    label = imagepath.split(os.path.sep)[-2]
    #print(label)
    image = cv2.imread(imagepath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    data.append(image)
    labels.append(label)

len(labels)

273

In [4]:
from sklearn.preprocessing import LabelBinarizer
data = np.array(data)
labels = np.array(labels)
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [5]:
from sklearn.model_selection import train_test_split
(X_train, X_test, y_train, y_test) = train_test_split(data, labels, test_size = 0.25, stratify = labels, random_state = 40)

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
traininAugmentation = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,horizontal_flip=True,fill_mode="nearest")

validationAugmentation = ImageDataGenerator()
mean = np.array([123.68, 116.779, 103.939], dtype = "float32")
traininAugmentation.mean = mean
validationAugmentation.mean = mean

ResNet_50

In [7]:
from keras.models import Model
from keras.layers import Dense, AveragePooling2D
from tensorflow.keras.applications import ResNet50
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout

In [13]:
resnet = ResNet50(weights = 'imagenet', input_shape = (224, 224, 3), include_top = False)
resnet_last_layer = AveragePooling2D(pool_size=(7,7))(resnet.output)
resnet_last_layer = Flatten()(resnet_last_layer)
resnet_last_layer = Dense(1024, activation='relu')(resnet_last_layer)
resnet_last_layer = Dropout(0.5)(resnet_last_layer)
resnet_last_layer = Dense(1024, activation='relu')(resnet_last_layer)
resnet_last_layer = Dropout(0.3)(resnet_last_layer)
resnet_last_layer = Dense(512, activation='relu')(resnet_last_layer)
resnet_last_layer = Dropout(0.2)(resnet_last_layer)
resnet_last_layer = Dense(2, activation='sigmoid')(resnet_last_layer)
model = Model(inputs = resnet.input, outputs = resnet_last_layer)

for layer in resnet.layers:
  layer.trainable = False

In [14]:
from tensorflow.keras.optimizers import Adam, SGD
adam = Adam(learning_rate = 0.0001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08)
sgd = SGD(learning_rate = 0.0001, momentum = 0.9, decay = 1e-4/20)

In [15]:
model.compile(loss="sparse_categorical_crossentropy", optimizer = adam, metrics = ["accuracy"])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('best_model_resnet.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [16]:
history = model.fit_generator(
    traininAugmentation.flow(X_train, y_train, batch_size=32),
    steps_per_epoch = len(X_train) // 32,
    validation_data = validationAugmentation.flow(X_test,y_test),
    validation_steps = len(X_test) // 32,
    epochs = 25, callbacks = [es, mc])

C:\Users\Shubham Satpute\AppData\Local\Temp\ipykernel_20688\1955056854.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/25
6/6 [==============================] - ETA: 0s - loss: 0.7507 - accuracy: 0.2760
Epoch 00001: val_accuracy improved from -inf to 0.42188, saving model to best_model_resnet.h5
6/6 [==============================] - 28s 4s/step - loss: 0.7507 - accuracy: 0.2760 - val_loss: 0.4139 - val_accuracy: 0.4219
Epoch 2/25
6/6 [==============================] - ETA: 0s - loss: 0.4624 - accuracy: 0.4826
Epoch 00002: val_accuracy improved from 0.42188 to 0.45312, saving model to best_model_resnet.h5
6/6 [==============================] - 22s 4s/step - loss: 0.4624 - accuracy: 0.4826 - val_loss: 0.3431 - val_accuracy: 0.4531
Epoch 3/25
6/6 [==============================] - ETA: 0s - loss: 0.3805 - accuracy: 0.4479
Epoch 00003: val_accuracy did not improve from 0.45312
6/6 [==============================] - 23s 4s/step - loss: 0.3805 - accuracy: 0.4479 - val_loss: 0.2718 - val_accuracy: 0.4219
Epoch 4/25
6/6 [==============================] - ETA: 0s - loss: 0.3952 - accuracy: 0.4186
Epoch

In [17]:
model.evaluate(X_test, y_test)

3/3 [==============================] - 6s 1s/step - loss: 0.2826 - accuracy: 0.4203


[0.28255805373191833, 0.4202898442745209]

VGG_19

In [18]:
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.applications.vgg19 import VGG19
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout

In [19]:
vgg19 = VGG19(weights = 'imagenet', input_shape = (224, 224, 3), include_top = False)
vgg_last_layer = GlobalAveragePooling2D()(vgg19.output)
vgg_last_layer = Flatten()(vgg_last_layer)
vgg_last_layer = Dense(1024, activation='relu')(vgg_last_layer)
vgg_last_layer = Dropout(0.5)(vgg_last_layer)
vgg_last_layer = Dense(1024, activation='relu')(vgg_last_layer)
vgg_last_layer = Dropout(0.3)(vgg_last_layer)
vgg_last_layer = Dense(512, activation='relu')(vgg_last_layer)
vgg_last_layer = Dropout(0.2)(vgg_last_layer)
vgg_last_layer = Dense(2, activation='sigmoid')(vgg_last_layer)
model = Model(inputs = vgg19.input, outputs = vgg_last_layer)

for layer in vgg19.layers:
  layer.trainable = False

#mobile = MobileNet(input_shape=(224,224,3), weights='imagenet', include_top=False)
#vgg = tf.keras.applications.vgg16.VGG16(input_shape=(224,224,3), weights='imagenet', include_top=False)

80150528/80134624 [==============================] - 29s 0us/step


In [20]:
from tensorflow.keras.optimizers import Adam
adam = Adam(learning_rate = 0.0001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08)

In [21]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = adam, metrics = ["accuracy"])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [22]:
#model.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics = ["accuracy"])
history = model.fit_generator(
    traininAugmentation.flow(X_train, y_train, batch_size=32),
    steps_per_epoch = len(X_train) // 32,
    validation_data = validationAugmentation.flow(X_test,y_test),
    validation_steps = len(X_test) // 32,
    epochs = 8, callbacks = [es, mc])

C:\Users\Shubham Satpute\AppData\Local\Temp\ipykernel_20688\472305448.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/8
6/6 [==============================] - ETA: 0s - loss: 1.8915 - accuracy: 0.5000
Epoch 00001: val_accuracy improved from -inf to 0.40625, saving model to best_model.h5
6/6 [==============================] - 53s 8s/step - loss: 1.8915 - accuracy: 0.5000 - val_loss: 0.3662 - val_accuracy: 0.4062
Epoch 2/8
6/6 [==============================] - ETA: 0s - loss: 1.1211 - accuracy: 0.3779
Epoch 00002: val_accuracy improved from 0.40625 to 0.43750, saving model to best_model.h5
6/6 [==============================] - 53s 10s/step - loss: 1.1211 - accuracy: 0.3779 - val_loss: 0.3355 - val_accuracy: 0.4375
Epoch 3/8
6/6 [==============================] - ETA: 0s - loss: 0.9520 - accuracy: 0.4896
Epoch 00003: val_accuracy did not improve from 0.43750
6/6 [==============================] - 57s 10s/step - loss: 0.9520 - accuracy: 0.4896 - val_loss: 0.3785 - val_accuracy: 0.4219
Epoch 4/8
6/6 [==============================] - ETA: 0s - loss: 0.7989 - accuracy: 0.4070
Epoch 00004: val_accu

In [23]:
model.evaluate(X_test, y_test)

3/3 [==============================] - 14s 3s/step - loss: 0.3956 - accuracy: 0.4203


[0.39558184146881104, 0.4202898442745209]

MobileNet

In [24]:
from keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from keras.layers.pooling import GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

In [25]:
mobile_net = MobileNet(weights = 'imagenet', input_shape = (224, 224, 3), include_top = False)
mn_last_layer = GlobalAveragePooling2D()(mobile_net.output)
mn_last_layer = Flatten()(mn_last_layer)
mn_last_layer = Dense(1024, activation='relu')(mn_last_layer)
mn_last_layer = Dropout(0.5)(mn_last_layer)
mn_last_layer = Dense(1024, activation='relu')(mn_last_layer)
mn_last_layer = Dropout(0.3)(mn_last_layer)
mn_last_layer = Dense(512, activation='relu')(mn_last_layer)
mn_last_layer = Dropout(0.2)(mn_last_layer)
mn_last_layer = Dense(2, activation='sigmoid')(mn_last_layer)
model = Model(inputs = mobile_net.input, outputs = mn_last_layer)

for layer in mobile_net.layers:
  layer.trainable = False

17235968/17225924 [==============================] - 9s 1us/step


In [26]:
from tensorflow.keras.optimizers import Adam
adam = Adam(learning_rate = 0.0001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08)

In [27]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = adam, metrics = ["accuracy"])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [28]:
history = model.fit_generator(
    traininAugmentation.flow(X_train, y_train, batch_size=32),
    steps_per_epoch = len(X_train) // 32,
    validation_data = validationAugmentation.flow(X_test,y_test),
    validation_steps = len(X_test) // 32,
    epochs = 100, callbacks = [es, mc])

C:\Users\Shubham Satpute\AppData\Local\Temp\ipykernel_20688\85072829.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/100
6/6 [==============================] - ETA: 0s - loss: 0.6633 - accuracy: 0.3256
Epoch 00001: val_accuracy improved from -inf to 0.45312, saving model to best_model.h5
6/6 [==============================] - 9s 1s/step - loss: 0.6633 - accuracy: 0.3256 - val_loss: 0.5098 - val_accuracy: 0.4531
Epoch 2/100
6/6 [==============================] - ETA: 0s - loss: 0.5999 - accuracy: 0.4323
Epoch 00002: val_accuracy did not improve from 0.45312
6/6 [==============================] - 7s 1s/step - loss: 0.5999 - accuracy: 0.4323 - val_loss: 0.3649 - val_accuracy: 0.3906
Epoch 3/100
6/6 [==============================] - ETA: 0s - loss: 0.4286 - accuracy: 0.4535
Epoch 00003: val_accuracy did not improve from 0.45312
6/6 [==============================] - 6s 1s/step - loss: 0.4286 - accuracy: 0.4535 - val_loss: 0.2771 - val_accuracy: 0.3750
Epoch 4/100
6/6 [==============================] - ETA: 0s - loss: 0.4205 - accuracy: 0.4012
Epoch 00004: val_accuracy did not improve from 0.4531

In [29]:
saved_model = load_model('best_model.h5')

In [30]:
model.evaluate(X_test, y_test)

3/3 [==============================] - 2s 465ms/step - loss: 0.2830 - accuracy: 0.3913


[0.28295931220054626, 0.3913043439388275]